In [4]:
from dotenv import load_dotenv, find_dotenv, dotenv_values
from langchain_openai import ChatOpenAI
import os

def hide_api_key(s):
    if len(s) < 6:
        return "Input string must be at least 6 characters long"
    return s[:3] + '...' + s[-3:]

dotenv_path = find_dotenv(filename=".env", raise_error_if_not_found=True)
load_dotenv(dotenv_path, override=True)

env_vars = dotenv_values(dotenv_path)
for k, v in env_vars.items():
    if "OPENAI_API_KEY" in k:
        print(f"{k}={hide_api_key(v)}")
    else:
        print(f"{k}={v}")


chat_model = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

# DEBUG: Print all environment variables
# for k, v in os.environ.items():
#     print(f"{k}={v}")

OPENAI_API_KEY=sk-...TBW


In [18]:
from langchain.schema.messages import HumanMessage, SystemMessage

messages = [
    SystemMessage(
        content="""You're an assistant knowledgeable about healthcare.
        Only answer healthcare-related questions."""
    ),
    HumanMessage(content="What is the best way to treat a cold?"),
]

chat_model.invoke(messages)

AIMessage(content="The common cold is a viral infection, so antibiotics are not effective in treating it. The best way to treat a cold is to get plenty of rest, stay hydrated, and manage symptoms with over-the-counter medications such as decongestants, pain relievers, and cough suppressants. It's also important to practice good hygiene, such as washing your hands frequently and covering your mouth and nose when you cough or sneeze, to prevent spreading the virus to others. If symptoms persist or worsen, it's a good idea to consult with a healthcare provider.", response_metadata={'token_usage': {'completion_tokens': 114, 'prompt_tokens': 36, 'total_tokens': 150}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-e8ecdc7b-8ab0-4f06-b5db-b982f6da681e-0')

In [20]:
chat_model.invoke("What is blood pressure?")

AIMessage(content='Blood pressure is the force of blood pushing against the walls of the arteries as the heart pumps blood throughout the body. It is measured in millimeters of mercury (mmHg) and is typically expressed as two numbers, such as 120/80 mmHg. The first number (systolic pressure) represents the pressure in the arteries when the heart beats, while the second number (diastolic pressure) represents the pressure in the arteries when the heart is at rest between beats. High blood pressure, also known as hypertension, can increase the risk of heart disease, stroke, and other health problems.', response_metadata={'token_usage': {'completion_tokens': 123, 'prompt_tokens': 12, 'total_tokens': 135}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-8dc3806b-84f1-434d-bf30-4050c49fca8b-0')

Prompt Templates

In [22]:
from langchain.prompts import ChatPromptTemplate

review_template_str = """Your job is to use patient
reviews to answer questions about their experience at a hospital.
Use the following context to answer questions. Be as detailed
as possible, but don't make up any information that's not
from the context. If you don't know an answer, say you don't know.

{context}

{question}
"""

review_template = ChatPromptTemplate.from_template(review_template_str)

context = "I had a great stay!"
question = "Did anyone have a positive expirence?"

review_template.format(context=context, question=question)

"Human: Your job is to use patient\nreviews to answer questions about their experience at a hospital.\nUse the following context to answer questions. Be as detailed\nas possible, but don't make up any information that's not\nfrom the context. If you don't know an answer, say you don't know.\n\nI had a great stay!\n\nDis anyone have a positive expirence?\n"

(as it is above) `ChatPromptTemplate` assumes the string template is a human message by default.
Here is how to change it:

In [5]:
from langchain.prompts import (
    PromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    ChatPromptTemplate,
)

review_system_template_str = """Your job is to use patient
reviews to answer questions about their experience at a
hospital. Use the following context to answer questions.
Be as detailed as possible, but don't make up any information
that's not from the context. If you don't know an answer, say
you don't know.

{context}
"""

review_system_prompt = SystemMessagePromptTemplate(
    prompt=PromptTemplate(
        input_variables=["context"], template=review_system_template_str
    )
)

review_human_prompt = HumanMessagePromptTemplate(
    prompt=PromptTemplate(
        input_variables=["question"], template="{question}"
    )
)

messages = [review_system_prompt, review_human_prompt]
review_prompt_template = ChatPromptTemplate(
    input_variables=["context", "question"],
    messages=messages,
)

context = "I had a great stay!"
question = "Did anyone have a positive expirence?"

review_prompt_template.format_messages(context=context, question=question)

[SystemMessage(content="Your job is to use patient\nreviews to answer questions about their experience at a\nhospital. Use the following context to answer questions.\nBe as detailed as possible, but don't make up any information\nthat's not from the context. If you don't know an answer, say\nyou don't know.\n\nI had a great stay!\n"),
 HumanMessage(content='Did anyone have a positive expirence?')]

LCEL (LangChain Expression Language) is the way to glue prompt with model and abstract away all of the internal details:

In [9]:
chat_model = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

review_chain = review_prompt_template | chat_model


context = "I had a great stay!"
question = "Did anyone have a positive expirence?"

review_chain.invoke({"context": context, "question": question})

AIMessage(content='Yes, the patient had a great stay at the hospital, indicating a positive experience.', response_metadata={'token_usage': {'completion_tokens': 17, 'prompt_tokens': 88, 'total_tokens': 105}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-539f4c2e-5682-4d74-aedf-9b7884621f4a-0')

we can push futher with LCEL pipe example and pass output into parser

In [10]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

review_chain = review_prompt_template | chat_model | output_parser

review_chain.invoke({"context": context, "question": question})

'Yes, the patient who left the review had a great stay at the hospital.'